# Prediction of sales

### Problem Statement
[The dataset](https://drive.google.com/file/d/1B07fvYosBNdIwlZxSmxDfeAf9KaygX89/view?usp=sharing) represents sales data for 1559 products across 10 stores in different cities. Also, attributes of each product and store are available. The aim is to build a predictive model and determine the sales of each product at a particular store.

|Variable|Description|
|: ------------- |:-------------|
|Item_Identifier|Unique product ID|
|Item_Weight|Weight of product|
|Item_Fat_Content|Whether the product is low fat or not|
|Item_Visibility|The % of total display area of all products in a store allocated to the particular product|
|Item_Type|The category to which the product belongs|
|Item_MRP|Maximum Retail Price (list price) of the product|
|Outlet_Identifier|Unique store ID|
|Outlet_Establishment_Year|The year in which store was established|
|Outlet_Size|The size of the store in terms of ground area covered|
|Outlet_Location_Type|The type of city in which the store is located|
|Outlet_Type|Whether the outlet is just a grocery store or some sort of supermarket|
|Item_Outlet_Sales|Sales of the product in the particulat store. This is the outcome variable to be predicted.|

Please note that the data may have missing values as some stores might not report all the data due to technical glitches. Hence, it will be required to treat them accordingly.



### In following weeks, we will explore the problem in following stages:

1. **Hypothesis Generation – understanding the problem better by brainstorming possible factors that can impact the outcome**
2. **Data Exploration – looking at categorical & continuous feature summaries and making inferences about the data**
3. **Data Cleaning – imputing missing values in the data and checking for outliers**
4. **Feature Engineering – modifying existing variables and/or creating new ones for analysis**
5. **Model Building – making predictive models on the data**
---------

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [21]:
data = pd.read_csv('regression_exercise.csv')

In [22]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## 4. Feature Engineering

1. Resolving the issues in the data to make it ready for the analysis.
2. Create some new variables using the existing ones.





### Create a broad category of Type of Item

`Item_Type` variable has many categories which might prove to be very useful in analysis. Look at the `Item_Identifier`, i.e. the unique ID of each item, it starts with either FD, DR or NC. If you see the categories, these look like being Food, Drinks and Non-Consumables. 

**Task:** Use the Item_Identifier variable to create a new column

In [23]:
# Extract the first two characters from 'Item_Identifier'
data['Item_Type_Broad'] = data['Item_Identifier'].str[:2]

# Define a mapping from acronyms to categories
acronym_to_category = {'FD': 'Food', 'DR': 'Drink', 'NC': 'Non-Consumable'}

# Replace the acronyms with their corresponding categories
data['Item_Type_Broad'] = data['Item_Type_Broad'].map(acronym_to_category)

### Determine the years of operation of a store

**Task:** Make a new column depicting the years of operation of a store (i.e. how long the store exists). 

In [24]:
import datetime

# Get the current year
current_year = datetime.datetime.now().year

# Calculate years of operation
data['Years_of_Operation'] = current_year - data['Outlet_Establishment_Year']

### Modify categories of Item_Fat_Content

**Task:** There are difference in representation in categories of Item_Fat_Content variable. This should be corrected.

In [25]:
data['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [26]:
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'low fat': 'Low Fat', 'LF': 'Low Fat', 'reg': 'Regular'})

In [27]:
data['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

**Task:** There are some non-consumables as well and a fat-content should not be specified for them. Create a separate category for such kind of observations.

In [28]:
# Replace 'Item_Fat_Content' values for non-consumables with 'Not Applicable'
data.loc[data['Item_Type_Broad'] == 'Non-Consumable', 'Item_Fat_Content'] = 'Not Applicable'

### Numerical and One-Hot Encoding of Categorical variables

Since scikit-learn algorithms accept only numerical variables, we need to **convert all categorical variables into numeric types.** 

- if the variable is Ordinal we can simply map its values into numbers
- if the variable is Nominal (we cannot sort the values) we need to One-Hot Encode them --> create dummy variables

In [29]:
# Define mappings for ordinal variables
outlet_size_mapping = {'Small': 1, 'Medium': 2, 'High': 3}
outlet_location_type_mapping = {'Tier 3': 1, 'Tier 2': 2, 'Tier 1': 3}  # adjust this mapping based on your understanding of the ordinality

# Replace the categories with numbers
data['Outlet_Size'] = data['Outlet_Size'].map(outlet_size_mapping)
data['Outlet_Location_Type'] = data['Outlet_Location_Type'].map(outlet_location_type_mapping)

In [30]:
# List of variables to be one-hot encoded
vars_to_encode = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Type', 'Item_Type_Broad']

# Perform one-hot encoding
data = pd.get_dummies(data, columns=vars_to_encode)

In [31]:
data

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,Years_of_Operation,Item_Fat_Content_Low Fat,...,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Type_Broad_Drink,Item_Type_Broad_Food,Item_Type_Broad_Non-Consumable
0,FDA15,9.300,0.016047,249.8092,1999,2.0,3,3735.1380,25,True,...,False,False,True,False,True,False,False,False,True,False
1,DRC01,5.920,0.019278,48.2692,2009,2.0,1,443.4228,15,False,...,False,False,False,False,False,True,False,True,False,False
2,FDN15,17.500,0.016760,141.6180,1999,2.0,3,2097.2700,25,True,...,False,False,True,False,True,False,False,False,True,False
3,FDX07,19.200,0.000000,182.0950,1998,NaN,1,732.3800,26,False,...,False,False,False,True,False,False,False,False,True,False
4,NCD19,8.930,0.000000,53.8614,1987,3.0,1,994.7052,37,False,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,0.056783,214.5218,1987,3.0,1,2778.3834,37,True,...,False,False,False,False,True,False,False,False,True,False
8519,FDS36,8.380,0.046982,108.1570,2002,NaN,2,549.2850,22,False,...,True,False,False,False,True,False,False,False,True,False
8520,NCJ29,10.600,0.035186,85.1224,2004,1.0,2,1193.1136,20,False,...,False,False,False,False,True,False,False,False,False,True
8521,FDN46,7.210,0.145221,103.1332,2009,2.0,1,1845.5976,15,False,...,False,False,False,False,False,True,False,False,True,False


**All variables should be by now numeric.**

---------
### Exporting Data

**Task:** You can save the processed data to your local machine as a csv file.

In [32]:
data.to_csv('data.csv', index=False)